In [1]:
import requests
# If you are using a Jupyter notebook, uncomment the following line.
#%matplotlib inline
import matplotlib.pyplot as plt
from io import BytesIO
import numpy as np
import cv2

#  ms_faceAPI를 사용하여 emotion dict로 추출
# 이미지 dir path를 넣어주면 됨
def face_classification(image_path):
    subscription_key = "173bd422f7a94a07887cd4a29b3c29fb"
    assert subscription_key

# You must use the same region in your REST call as you used to get your
# subscription keys. For example, if you got your subscription keys from
# westus, replace "westcentralus" in the URI below with "westus".
#
# Free trial subscription keys are generated in the westcentralus region.
# If you use a free trial subscription key, you shouldn't need to change
# this region.
    vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect"

# Set image_path to the local path of an image that you want to analyze.
#     image_path = "image/lena.jpg"

# Read the image into a byte array
    try:
        image_path1 = image_path +'.jpg'
        image_data = open(image_path1, "rb").read()
    except:
        image_path2 = image_path +'.png'
        image_data = open(image_path2, "rb").read()


    headers    = {'Ocp-Apim-Subscription-Key': subscription_key,
              'Content-Type': 'application/octet-stream'}
    params = {
    'returnFaceId': 'false',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'emotion'
}
    response = requests.post(
    vision_base_url, headers=headers, params=params, data=image_data)
    response.raise_for_status()

# The 'analysis' object contains various fields that describe the image. The most
# relevant caption for the image is obtained from the 'description' property.
    analysis = response.json()
    emotion = analysis[0]['faceAttributes']['emotion']
    return emotion


def save_emotion(image_path,emotion,name):
    
    try:
        image_path1 = image_path +'.jpg'
        frame = cv2.imread(image_path1, cv2.IMREAD_COLOR)
    except:
        image_path2 = image_path +'.png'
        frame = cv2.imread(image_path2, cv2.IMREAD_COLOR)

    
    #data 정제(score, class)
    emotion_list = [(k,emotion[k]) for k in emotion]
    emotion_score = [emotion[k] for k in emotion]
    emotion_class = [k for k in emotion]
    

    emotion_score = np.array(emotion_score)
    selected_class = emotion_class[np.argmax(emotion_score)]

    cv2.imwrite('D:\workspace/emotion/'+selected_class+'/'+name+'b.jpg',frame)


In [2]:
for i in range(1,26532):
    image_name = 'i ('+str(i)+')'
    #파일 디렉토리와 이름 지정
    image_path = 'D:\workspace/data/'+image_name
    try:
        emotion = face_classification(image_path)
    except:
        continue
    
    save_emotion(image_path,emotion,str(i))